In [1]:
import csv
import math
import os
from copy import copy, deepcopy

In [1]:
notebook_path = os.path.abspath(os.path.join(os.path.dirname(os.path.abspath("lin_regr.ipynb")), os.pardir))
data_folder = "/data/"

In [3]:
airline_delay_causes = []
columns = []
target_variable = "DEP_DELAY"
target_col = []

In [4]:
with open(notebook_path+data_folder+'2017.csv', newline='\n') as csv_file:
	csv_reader = csv.reader(csv_file, delimiter=',')
	line_count = 0
	for row in csv_reader:
		if line_count == 0:
			columns = row.copy()
			line_count += 1
		else:
			airline_delay_causes.append(row)
			line_count += 1

In [5]:
target_index = 11
# features = [1, 2, 9, 10, 12, 17, 19, 25, 26, 27, 28, 29]
features = [9, 25, 29]
data_train_features = []
data_train_target = []
for row in airline_delay_causes:
	tmp = []
	for col in features:
		if row[col] == "":
			break
		else:
			tmp.append(float(row[col]))
	if len(tmp) == len(features):
		data_train_features.append(tmp)
		data_train_target.append(float(row[target_index]))

In [6]:
def norm(V):
    ans = 0
    for i in V:
        ans += i**2
    return math.sqrt(ans)

def sumvc(X, a):
    ans = X.copy()
    for i in range(len(X)):
        ans[i] += a
    return ans

def sumvv(X, Y):
    if len(X) != len(Y):
        raise Exception("Len of vectors should be same!")
    ans = X.copy()
    for i in range(len(Y)):
        ans[i] += Y[i]
    return ans

def subvc(X, a):
    ans = X.copy()
    for i in range(len(X)):
        ans[i] -= a
    return ans

def subvv(X, Y):
    if len(X) != len(Y):
        raise Exception("Len of vectors should be same!")
    ans = X.copy()
    for i in range(len(Y)):
        ans[i] -= Y[i]
    return ans

def multvc(X, c):
    ans = X.copy()
    for i in range(len(X)):
        ans[i] *= c
    return ans

def multvv(X, Y):
    if len(X) != len(Y):
        raise Exception("Len of vectors should be same!")
    ans = X.copy()
    for i in range(len(Y)):
        ans[i] *= Y[i]
    return ans

def dotvv(X, Y):
    if len(X) != len(Y):
        raise Exception("Len of vectors should be same!")
    ans = 0
    for i in range(len(X)):
        ans += X[i] * Y[i]
    return ans

def summc(A, c):
    if len(A) == 0:
        return []
    rows_A = len(A)
    cols_A = len(A[0])
    ans = [[0 for row in range(cols_A)] for col in range(rows_A)]
    for i in range(rows_A):
        for j in range(cols_A):
            ans[i][j] = A[i][j] + c
    return ans

def summm(A, B):
    if len(A) == 0 or len(B) == 0:
        return []
    if len(A) != len(B) or len(A[0]) != len(B[0]):
        raise Exception("Dim of matrix should be same!")
    rows_A = len(A)
    cols_B = len(B[0])
    ans = [[0 for row in range(cols_B)] for col in range(rows_A)]
    for i in range(rows_A):
        for j in range(cols_B):
            ans[i][j] = A[i][j] + B[i][j]
    return ans

def submc(A, c):
    if len(A) == 0:
        return []
    rows_A = len(A)
    cols_A = len(A[0])
    ans = [[0 for row in range(cols_A)] for col in range(rows_A)]
    for i in range(rows_A):
        for j in range(cols_A):
            ans[i][j] = A[i][j] - c
    return ans

def submm(A, B):
    if len(A) == 0 or len(B) == 0:
        return []
    if len(A) != len(B) or len(A[0]) != len(B[0]):
        raise Exception("Dim of matrix should be same!")
    rows_A = len(A)
    cols_B = len(B[0])
    ans = [[0 for row in range(cols_B)] for col in range(rows_A)]
    for i in range(rows_A):
        for j in range(cols_B):
            ans[i][j] = A[i][j] - B[i][j]
    return ans

def multmc(A, c):
    if len(A) == 0:
        return []
    rows_A = len(A)
    cols_A = len(A[0])
    ans = [[0 for col in range(cols_A)] for row in range(rows_A)]
    for i in range(rows_A):
        for j in range(cols_A):
            ans[i][j] = A[i][j] * c
    return ans

def multmv(A, V):
    if len(A) == 0:
        return []
    rows_A = len(A)
    cols_A = len(A[0])
    rows_V = len(V)
    if cols_A != rows_V:
        raise Exception("Cannot multiply the matrix and vector. Incorrect dimensions.")
    ans = [0 for row in range(rows_A)]
    for i in range(rows_A):
        for j in range(cols_A):
            ans[i] += A[i][j] * V[j]
    return ans

def multmm(A, B):
    if len(A) == 0 or len(B) == 0:
        return []
    rows_A = len(A)
    cols_A = len(A[0])
    rows_B = len(B)
    cols_B = len(B[0])
    if cols_A != rows_B:
        raise Exception("Cannot multiply the two matrices. Incorrect dimensions.")
    ans = [[0 for row in range(cols_B)] for col in range(rows_A)]
    for i in range(rows_A):
        for j in range(cols_B):
            for k in range(cols_A):
                ans[i][j] += A[i][k] * B[k][j]
    return ans

def decvc(V, c):
    ans = V.copy()
    for i in range(len(V)):
        ans[i] /= c
    return ans

def decvv(V, W):
    ans = V.copy()
    for i in range(len(V)):
        ans[i] /= W[i]
    return ans

def sqrtv(V):
    ans = V.copy()
    for i in range(len(V)):
        ans[i] = math.sqrt(ans[i])
    return ans

def powv(V, s):
    if s == 0:
        return [1 for el in range(len(V))]
    tmp = powv(V, int(s/2))
    if s % 2 == 0:
        return multvv(tmp, tmp)
    else:
        return multvv(multvv(tmp, tmp), V)

In [7]:
def MSE(W, X, Y):
    if len(X[0]) != len(W) or len(X) != len(Y):
        raise Exception("Len of vectors should be same!")
    ans = 0
    for i in range(len(X)):
        ans += ((dotvv(W, X[i]) - Y[i])**2)/len(X)
    return ans

In [8]:
def R2(W, X, Y):
	if len(X[0]) != len(W) or len(X) != len(Y):
		raise Exception("Len of vectors should be same!")
	ym = sum(Y)/len(Y)
	SSreg = 0
	for i in range(len(X)):
		SSreg += (dotvv(W, X[i]) - ym)**2
	SStot = 0
	for i in range(len(Y)):
		SStot += (Y[i] - ym)**2
	return 1 - (SSreg / SStot)

In [9]:
def grad(fun, h, W, X, Y):
	col_X = len(X[0])
	ans = [0 for i in range(col_X)]
	Wn = W.copy()
	for i in range(col_X):
		Wn[i] += h
		ans[i] = (MSE(Wn, X, Y) - MSE(W, X, Y))/h
		Wn[i] -= h
	return ans

In [10]:
def LeastSquareMethod(A, y):
    col_A = len(A[0])
    W = [1. for row in range(col_A)]
    step = 1e-8
    h = 1e-8
    L = 0
    Lnew = MSE(W, A, y)
    num_steps = 200
    while abs(L - Lnew) > 1e-6 and num_steps > 0:
        L = Lnew
        W = subvv(W, multvc(grad(MSE, h, W, A, y), step))
        Lnew = MSE(W, A, y)
        num_steps -= 1
    return W

In [11]:
def SGD(A, y):
	col_A = len(A[0])
	W = [1. for row in range(col_A)]
	step = 1e-8
	h = 1e-8
	L = 0
	Lnew = MSE(W, A, y)
	num_steps = 50
	while abs(L - Lnew) > 1e-5 and num_steps > 0:
		for i in range(len(A)):
			L = Lnew
			W = subvv(W, multvc(grad(MSE, h, W, [A[i]], [y[i]]), step))
			Lnew = MSE(W, A, y)
		num_steps -= 1
	return W

In [20]:
def AdaGrad(A, y):
	col_A = len(A[0])
	W = [1. for row in range(col_A)]
	step = 1e-1
	h = 1e-2
	eps = 1
	L = MSE(W, A, y)
	Gnii = [0 for i in range(col_A)]
	temp = grad(MSE, h, W, A, y)
	for i in range(col_A):
		Gnii[i] = (temp[i])**2
		W[i] -= step*temp[i]/math.sqrt(Gnii[i] + eps)
	Lnew = MSE(W, A, y)
	num_steps = 2500
	while abs(L - Lnew) > 1e-1 and num_steps > 0:
		L = Lnew
		temp = grad(MSE, h, W, A, y)
		for i in range(col_A):
			Gnii[i] += (temp[i])**2
			W[i] -= step*temp[i]/math.sqrt(Gnii[i] + eps)
		Lnew = MSE(W, A, y)
		num_steps -= 1
	return W

In [18]:
def RMSProp(A, y, gamma=0.5):
    col_A = len(A[0])
    W = [1. for row in range(col_A)]
    step = 1e-3
    h = 1e-3
    eps = 1e-3
    L = MSE(W, A, y)
    Eg = 0
    tmp = 0
    tmpnew = grad(MSE, h, W, A, y)
    Eg = multvc(powv(tmpnew, 2), 1-gamma)
    W = subvv(W, decvv(multvc(tmpnew, step), sqrtv(sumvc(Eg, eps))))
    Lnew = MSE(W, A, y)
    num_steps = 3500
    while abs(L - Lnew) > 1e-2 and num_steps > 0:
        L = Lnew
        tmp = tmpnew
        tmpnew = grad(MSE, h, W, A, y)
        Eg = sumvv(multvc(powv(tmp, 2), gamma), multvc(powv(tmpnew, 2), 1-gamma))
        W = subvv(W, decvv(multvc(tmpnew, step), sqrtv(sumvc(Eg, eps))))
        Lnew = MSE(W, A, y)
        num_steps -= 1
    return W

In [27]:
def Adam(A, y, gamma=0.9, beta=[0.95, 0.95]):
	col_A = len(A[0])
	W =[1. for row in range(col_A)]
	step = 1e-4
	h = 1e-3
	eps = 1e-3
	L = 0
	Lnew = MSE(W, A, y)
	tmp = grad(MSE, h, W, A, y)
	mt = multvc(tmp, 1-beta[0])
	_mt = decvc(mt, 1-beta[0])
	vt = multvc(powv(tmp, 2), 1-beta[1])
	_vt = decvc(vt, 1-beta[1])
	W = subvv(W, decvv(multvc(_mt, step), sqrtv(sumvc(_vt, eps))))
	L = Lnew
	Lnew = MSE(W, A, y)
	num_steps = 4000
	while abs(L - Lnew) > 1e-3 and num_steps > 0:
		L = Lnew
		tmp = grad(MSE, h, W, A, y)
		mt = sumvv(multvc(mt, beta[0]), multvc(tmp, 1-beta[0]))
		_mt = decvc(mt, 1-beta[0])
		vt = sumvv(multvc(vt, beta[1]), multvc(powv(tmp, 2), 1-beta[1]))
		_vt = decvc(vt, 1-beta[1])
		W = subvv(W, decvv(multvc(_mt, step), sqrtv(sumvc(_vt, eps))))
		Lnew = MSE(W, A, y)
		num_steps -= 1
	return W

In [14]:
W = LeastSquareMethod(data_train_features[:1000], data_train_target[:1000])
print("\nW (LSM) = ", W)
print("LSM MSE (train) = ", MSE(W, data_train_features[:1000], data_train_target[:1000]))
print("LSM R2  (train) = ", R2(W, data_train_features[:1000], data_train_target[:1000]))
print("LSM MSE (test) = ", MSE(W, data_train_features[1000:2000], data_train_target[1000:2000]))
print("LSM R2  (test) = ", R2(W, data_train_features[1000:2000], data_train_target[1000:2000]))


W (LSM) =  [0.013724120248753024, 0.9907197258683027, 0.9753648554094525]
LSM MSE (train) =  1957.3957065015277
LSM R2  (train) =  0.1596560069216585
LSM MSE (test) =  601.4375092979024
LSM R2  (test) =  0.10267201576114804


In [20]:
W = SGD(data_train_features[:1000], data_train_target[:1000])
print("\nW (SGD) = ", W)
print("SGD MSE (train) = ", MSE(W, data_train_features[:1000], data_train_target[:1000]))
print("SGD R2  (train) = ", R2(W, data_train_features[:1000], data_train_target[:1000]))
print("SGD MSE (test) = ", MSE(W, data_train_features[1000:2000], data_train_target[1000:2000]))
print("SGD R2  (test) = ", R2(W, data_train_features[1000:2000], data_train_target[1000:2000]))


W (SGD) =  [0.011597486216464739, 0.962417503753627, 0.9635366294544346]
SGD MSE (train) =  1971.866598471285
SGD R2  (train) =  0.18567889133635362
SGD MSE (test) =  550.3880396027234
SGD R2  (test) =  0.15129692712934517


In [21]:
W = AdaGrad(data_train_features[:1000], data_train_target[:1000])
print("\nW (AdaGrad) = ", W)
print("AdaGrad MSE (train) = ", MSE(W, data_train_features[:1000], data_train_target[:1000]))
print("AdaGrad R2  (train) = ", R2(W, data_train_features[:1000], data_train_target[:1000]))
print("AdaGrad MSE (test) = ", MSE(W, data_train_features[1000:2000], data_train_target[1000:2000]))
print("AdaGrad R2  (test) = ", R2(W, data_train_features[1000:2000], data_train_target[1000:2000]))


W (AdaGrad) =  [0.010389103142406984, 0.9472136142063095, 0.9337352738430533]
AdaGrad MSE (train) =  2017.5507011309812
AdaGrad R2  (train) =  0.22331541516980735
AdaGrad MSE (test) =  543.7920736730596
AdaGrad R2  (test) =  0.188470941943237


In [19]:
W = RMSProp(data_train_features[:1000], data_train_target[:1000])
print("\nW (RMSProp) = ", W)
print("RMSProp MSE (train) = ", MSE(W, data_train_features[:1000], data_train_target[:1000]))
print("RMSProp R2  (train) = ", R2(W, data_train_features[:1000], data_train_target[:1000]))
print("RMSProp MSE (test) = ", MSE(W, data_train_features[1000:2000], data_train_target[1000:2000]))
print("RMSProp R2  (test) = ", R2(W, data_train_features[1000:2000], data_train_target[1000:2000]))


W (RMSProp) =  [0.012906292593697863, 0.9601072131389291, 0.9739980460597351]
RMSProp MSE (train) =  1957.3283473464203
RMSProp R2  (train) =  0.1758587911345253
RMSProp MSE (test) =  577.5070306049705
RMSProp R2  (test) =  0.14401052531436365


In [31]:
W = Adam(data_train_features[:1000], data_train_target[:1000], gamma=0.85, beta=[0.8, 0.85])
print("\nW (Adam) = ", W)
print("Adam MSE (train) = ", MSE(W, data_train_features[:1000], data_train_target[:1000]))
print("Adam R2  (train) = ", R2(W, data_train_features[:1000], data_train_target[:1000]))
print("Adam MSE (test) = ", MSE(W, data_train_features[1000:2000], data_train_target[1000:2000]))
print("Adam R2  (test) = ", R2(W, data_train_features[1000:2000], data_train_target[1000:2000]))


W (Adam) =  [0.225990686292005, 0.2261867190555661, 0.22614563066087676]
Adam MSE (train) =  93228.74027138193
Adam R2  (train) =  -8.568488634117097
Adam MSE (test) =  98369.87465848193
Adam R2  (test) =  -14.694846543848852
